In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r'C:\Users\foxgh\OneDrive\Área de Trabalho\SQL Playground\Script\data.db')
cursor = conn.cursor()

In [11]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("Tabelas no banco de dados:", tabelas)


Tabelas no banco de dados: [('vendas',), ('estoque',)]


In [14]:
query = "select * from vendas"

df = pd.read_sql_query(query, conn)

# conn.close()

df.head()

,ID,Produto,Categoria,Preço,Quantidade_Vendida,Data_Venda,Loja,Vendedor
0,1,Produto_1,Móveis,391.84,42,2023-01-01,Loja E,Vendedor_8
1,2,Produto_2,Brinquedos,84.00,13,2023-01-02,Loja E,Vendedor_15
2,3,Produto_3,Móveis,306.32,35,2023-01-03,Loja C,Vendedor_19
3,4,Produto_4,Alimentos,124.38,32,2023-01-04,Loja A,Vendedor_17
4,5,Produto_5,Móveis,378.73,23,2023-01-05,Loja B,Vendedor_8


In [53]:
query = "select * from estoque"

df = pd.read_sql_query(query, conn)

# conn.close()

df.head()

,ID,Produto,Categoria,Estoque_Inicial,Estoque_Atual,Fornecedor,Data_Reabastecimento
0,1,Produto_1,Móveis,264,222,Fornecedor_7,2023-01-01
1,2,Produto_2,Brinquedos,445,432,Fornecedor_10,2023-01-16
2,3,Produto_3,Móveis,213,178,Fornecedor_8,2023-01-31
3,4,Produto_4,Alimentos,407,375,Fornecedor_4,2023-02-15
4,5,Produto_5,Móveis,431,408,Fornecedor_8,2023-03-02


In [49]:
query = """ 
    with t1 as (
        select
            categoria,
            sum(preço) as Total,
            sum(quantidade_vendida) as quantidade
        from vendas
        group by categoria  
        order by quantidade desc
    )
    
    select  * from t1
"""

df = pd.read_sql_query(query, conn)
df

,categoria,Total,quantidade
0,Brinquedos,12874.36,1165
1,Alimentos,11573.70,1064
2,Móveis,9678.30,1059
3,Roupas,7425.15,909
4,Eletrônicos,9192.55,699


In [65]:
query = """ 
    with t1 as (
        select
            max(data_venda) as data_mais_recente,
            min(data_venda) as data_mais_antiga,
            produto,
            categoria,
            sum(preço) as total_vendas,
            sum(quantidade_vendida) as quantidade_vendida,
            sum(preço * quantidade_vendida) as total_receita,
            sum(preço) / sum(quantidade_vendida) as preço_medio,
            sum(preço * quantidade_vendida) / sum(quantidade_vendida) as receita_media
        from vendas
        group by produto, categoria
    )
    
    select  * from t1
"""

df = pd.read_sql_query(query, conn)
df

,data_mais_recente,data_mais_antiga,produto,categoria,total_vendas,quantidade_vendida,total_receita,preço_medio,receita_media
0,2023-01-01,2023-01-01,Produto_1,Móveis,391.84,42,16457.28,9.329524,391.84
1,2023-01-10,2023-01-10,Produto_10,Eletrônicos,449.59,32,14386.88,14.049687,449.59
2,2023-04-10,2023-04-10,Produto_100,Móveis,325.82,49,15965.18,6.649388,325.82
3,2023-04-11,2023-04-11,Produto_101,Móveis,169.64,37,6276.68,4.584865,169.64
4,2023-04-12,2023-04-12,Produto_102,Eletrônicos,192.23,15,2883.45,12.815333,192.23
...,...,...,...,...,...,...,...,...,...
195,2023-04-05,2023-04-05,Produto_95,Móveis,40.24,33,1327.92,1.219394,40.24
196,2023-04-06,2023-04-06,Produto_96,Eletrônicos,443.49,29,12861.21,15.292759,443.49
197,2023-04-07,2023-04-07,Produto_97,Brinquedos,341.87,16,5469.92,21.366875,341.87
198,2023-04-08,2023-04-08,Produto_98,Roupas,13.18,28,369.04,0.470714,13.18


In [54]:
query = """ 
    with t1 as (
        select 
            categoria, 
            sum(estoque_atual) as estoque_atual
        from estoque
        group by Categoria
    )
    
    select  * from t1
"""

df = pd.read_sql_query(query, conn)
df

,categoria,estoque_atual
0,Alimentos,12242
1,Brinquedos,12420
2,Eletrônicos,7110
3,Móveis,11143
4,Roupas,9156


In [ ]:
query = """ 
    with t1 as (
        select
            categoria,
            sum(preço) as Total,
            sum(quantidade_vendida) as quantidade
        from vendas
        group by categoria  
        order by quantidade desc
    ),
    
    t2 as (
        select 
            categoria, 
            sum(estoque_atual) as estoque_atual
        from estoque
        group by Categoria
    )
    
    select 
        t1.*,
        t2.estoque_atual
    from t1
    left join t2 on t1.categoria = t2.categoria
"""

df = pd.read_sql_query(query, conn)
df

,categoria,Total,quantidade,estoque_atual
0,Brinquedos,12874.36,1165,12420
1,Alimentos,11573.70,1064,12242
2,Móveis,9678.30,1059,11143
3,Roupas,7425.15,909,9156
4,Eletrônicos,9192.55,699,7110
